## Data606 - Capstone Project
```
Group H
Malav Patel, Kent Butler
Prof. Unal Sokaglu
```

This project is about performing time-series analysis on climate data analysis data.



# Research

### References

Some explanations of earth sciences statistics:
https://pjbartlein.github.io/REarthSysSci/ltms-and-anomalies.html

NOAA PSL NCEP-NCAR datasets:  https://psl.noaa.gov/data/gridded/data.ncep.reanalysis.html

NOAA PSL, other recognized data sources directory: https://psl.noaa.gov/data/help/othersources/

Global environmental policy timeline, https://www.boell.de/en/2022/05/28/international-environmental-policy-timeline

OECD convergence of policy, climate,and economy: https://www.oecd.org/

NASA climate time machine: https://climate.nasa.gov/interactives/climate-time-machine

### Factoids

* All of the plastic waste produced in the world in 2019 alone weighs as much as 35,000 Eiffel Towers – 353 million tons  - [*Organization for Economic Cooperation and Development (OECD)*](https://www.boell.de/en/2022/05/28/international-environmental-policy-timeline)



## Application Parameters

Note: algorithm tuning is done with declaration of the model.

In [ ]:
import pandas as pd
from datetime import datetime as dt
import datetime

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
plt.rcParams["figure.figsize"] = (10,6)
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn')
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
debug = False

DRIVE_PATH = "/content/drive/MyDrive/data606"

# Set the location of this script in GDrive
SCRIPT_PATH = DRIVE_PATH + "/src/"

# Root Path of the data on the cloud drive
DATA_ROOT = DRIVE_PATH + "/data/"

# Location of logged output prediction data
LOG_PATH = DATA_ROOT + "/preds/"

# Journal file
JOURNAL_LOG = SCRIPT_PATH + "cv-results.csv"

# Start including data from this date
START_DATE =  pd.to_datetime(dt.fromisoformat('1950-01-01'))
# Stop including data after this date
END_DATE = pd.to_datetime(dt.fromisoformat('2015-12-01'))

In [ ]:
%cd $SCRIPT_PATH

/content/drive/MyDrive/data606/src


In [ ]:
# Load util class
%run -i "./ProjectUtil.ipynb"

In [ ]:
# Load util class
%run -i "./ModelExecutor.ipynb"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/data606/src


In [ ]:
# Load model class
%run -i "./Model_LSTMv2.ipynb"

In [ ]:
# Load util class
#%run -i "./WindowGenerator.ipynb"

**Dataset Definitions**

In [ ]:
# Label to predict
TARGET_LABEL = 'landSeaAvgTemp'

# Base dataset
TEMP_DATA = {'filename':'GlobalTemperatures.csv',
             'feature_map':{'LandAndOceanAverageTemperature':'landSeaAvgTemp'},
             'date_col':'dt'}

#TEMP_DATA = {'filename':'GlobalTemperatures.csv',
#              'feature_map':{'LandAverageTemperature':'landAvgTemp',	'LandMaxTemperature':'landMaxTemp',	'LandMinTemperature':'landMinTemp',	'LandAndOceanAverageTemperature':'landSeaAvgTemp'},
#               'date_col':'dt'}

# Datasets
CO2_DATA = {'filename':"atmospheric-co2.csv",
            'feature_map':{'Carbon Dioxide (ppm)':'co2', 'Seasonally Adjusted CO2 (ppm)':'co2_seas'},
            'date_map':{'Year':'year','Month':'month'}}

SEAICE_DATA = {'filename':"seaice.csv",
               'feature_map':{'     Extent':'ice_extent','    Missing':'ice_missing'},
               'date_map':{' Month':'month','Year':'year',' Day':'day'}}

WEATHER_DATA = {'filename':"finalDatasetWithRain.csv",
                'feature_map':{'air_x':'air_x','air_y':'air_y','uwnd':'uwnd'},
                'date_col':'time'}

VOLCANO_DATA = {'filename':'eruptions-conditioned.csv',
                'feature_map':{'vei':'volcanic_idx'},
                'date_map':{'start_year':'year','start_month':'month'}}

FOREST_DATA = {'filename':'WorldForestCover.csv',
               'feature_map':{'PctCover':'pct_forest_cover'},
               'date_map':{'Year':'year'}}

SUNSPOT_DATA = {'filename':'sunspotnumber.csv',
               'feature_map':{'suns_spot_number':'sunspot_num'},
               'date_map':{'year':'year'}}

POLICY_DATA = {'filename':'GlobalEnvPolicies.csv',
               'feature_map':{'EventRating':'policy_rate'},
               'date_col':'date'}

#GHG_DATA = {'filename':'greenhouse_gas_inventory_data.csv',
#            'feature_map':{''},
#            'date_map':{'Year':'year'}}


**Hyperparams**

In [ ]:
# History lookback in network
INPUT_WINDOW = 30
# How far forward to predict
LABEL_WINDOW = 1
# Ratio of test data to train data - used for split
TEST_RATIO = 0.2
# 0..1 percent of data to use as validation
VALIDATION_RATIO = 0.1
# Num epochs
NUM_EPOCHS = 300
# Model to use
MODEL_NAME = "LSTMv2"

ALL_DATASETS = [[CO2_DATA],
  [CO2_DATA,FOREST_DATA],
  [CO2_DATA,SEAICE_DATA],
  [CO2_DATA,POLICY_DATA],
  [CO2_DATA,SEAICE_DATA,POLICY_DATA],
  [CO2_DATA,SEAICE_DATA,WEATHER_DATA],
  [CO2_DATA,SEAICE_DATA,WEATHER_DATA,FOREST_DATA],
  [CO2_DATA,SEAICE_DATA,WEATHER_DATA,FOREST_DATA,VOLCANO_DATA],
  [CO2_DATA,SEAICE_DATA,WEATHER_DATA,FOREST_DATA,VOLCANO_DATA,SUNSPOT_DATA],
  [CO2_DATA,VOLCANO_DATA],
  [CO2_DATA,VOLCANO_DATA,FOREST_DATA],
  [CO2_DATA,VOLCANO_DATA,FOREST_DATA,SEAICE_DATA],
  [CO2_DATA,VOLCANO_DATA,FOREST_DATA,SEAICE_DATA,SUNSPOT_DATA],
  [CO2_DATA,FOREST_DATA],
  [CO2_DATA,FOREST_DATA,SEAICE_DATA],
  [CO2_DATA,FOREST_DATA,SEAICE_DATA,SUNSPOT_DATA],
  [VOLCANO_DATA],
  [VOLCANO_DATA,FOREST_DATA],
  [VOLCANO_DATA,POLICY_DATA],
  [VOLCANO_DATA,FOREST_DATA,SUNSPOT_DATA],
  [VOLCANO_DATA,FOREST_DATA,SUNSPOT_DATA,POLICY_DATA],
  [VOLCANO_DATA,FOREST_DATA,SUNSPOT_DATA,POLICY_DATA,SEAICE_DATA],
  [FOREST_DATA],
  [FOREST_DATA,POLICY_DATA],
  [SEAICE_DATA],
  [SEAICE_DATA,FOREST_DATA],
  [SEAICE_DATA,POLICY_DATA],
  [SEAICE_DATA,FOREST_DATA,VOLCANO_DATA],
  [SEAICE_DATA,FOREST_DATA,VOLCANO_DATA,POLICY_DATA]
]

In [ ]:
#ALL_DATASETS=[ALL_DATASETS[1]]

# Execute Trainer

In [ ]:
# Declare a merger compatible with our source data and our target dataset we want to merge into
#  (self, data_path, journal_log, start_date, end_date, input_window, label_window, test_ratio, val_ratio, num_epochs, target_label, model_name, debug=False):

for i,ds_list in enumerate(ALL_DATASETS):

  fnames = [ds['filename'] for ds in ds_list]
  print(f'============================ Executing {i} ===================================\n{fnames}')

  # re-construct the model exec b/c it contains some state
  exec = ModelExecutor(data_path=DATA_ROOT, log_path=LOG_PATH, journal_log=JOURNAL_LOG, start_date=START_DATE, end_date=END_DATE,
                      input_window=INPUT_WINDOW, label_window=LABEL_WINDOW, test_ratio=TEST_RATIO, val_ratio=VALIDATION_RATIO,
                      num_epochs=NUM_EPOCHS, target_label=TARGET_LABEL, model_name=MODEL_NAME, debug=True)

  exec.load_initial_dataset(TEMP_DATA['filename'], TEMP_DATA['feature_map'], date_map=None, date_col=TEMP_DATA['date_col'])

  exec.load_datasets(ds_list)

  #exec.print_correlations()

  exec.process()

  print(f'=========================== Completed {i} ===================================')
